In [6]:
import pandas as pd
from pandas import Series,DataFrame
import numpy as np
import os
from sklearn.metrics import mean_squared_error
import datetime
import tensorflow as tf
from tensorflow.contrib import learn
from tensorflow.python.ops import rnn, rnn_cell
import matplotlib.pyplot as plt
import cPickle
import gzip
import math
%matplotlib inline

/usr/local/lib/python2.7/dist-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [7]:
# 提取主力合约数据
path = '/home/chocolate/LSTM-source/'
data = pd.read_csv(path + 'data_added_features.csv')
data=data.ix[:,3:]
data = data.fillna(0)
# data
# data['RaiseDown'] = (data['Latestprice'] - data['First.Latestprice'])/data['First.Latestprice']

In [8]:
# 为主力合约数据添加两个特征：平均价和下一分钟涨跌幅
# 其中，平均价为当前分钟的四个价格的平均值
# 下一分钟的涨跌幅为（下一分钟平均价 - 当前分钟平均价）/当前分钟平均价
data['MeanPrice'] = (data['Latestprice'] + data['First.Latestprice'] +data['MaxPrice']+data['MinPrice'])/4
ser_1 = data['MeanPrice'][1:]
ser_1.index = range(len(ser_1))
ser1 = data['MeanPrice'][:-1]
ser1.index = range(len(ser1))
data['RaiseDown_1'] = (ser_1 -ser1)/ser1
# 同理，计算当前分钟的后2分钟、3分钟、4分钟、5分钟的涨跌幅
ser_2 = data['MeanPrice'][2:]
ser_2.index = range(len(ser_2))
ser2 = data['MeanPrice'][:-2]
ser2.index = range(len(ser2))
data['RaiseDown_2'] = (ser_2 -ser2)/ser2

ser_3 = data['MeanPrice'][3:]
ser_3.index = range(len(ser_3))
ser3 = data['MeanPrice'][:-3]
ser3.index = range(len(ser3))
data['RaiseDown_3'] = (ser_3 -ser3)/ser3

ser_4 = data['MeanPrice'][4:]
ser_4.index = range(len(ser_4))
ser4 = data['MeanPrice'][:-4]
ser4.index = range(len(ser4))
data['RaiseDown_4'] = (ser_4 -ser4)/ser4

ser_5 = data['MeanPrice'][5:]
ser_5.index = range(len(ser_5))
ser5 = data['MeanPrice'][:-5]
ser5.index = range(len(ser5))
data['RaiseDown_5'] = (ser_5 -ser5)/ser5
data = data.fillna(0)

In [9]:
# 添加下一分钟的MA5、MA12、MA26、上轨线、下轨线指标
ser_1 = data['MA_5'][1:]
ser_1.index = range(len(ser_1))
data['MA_5_PRED'] = ser_1

ser_2 = data['MA_12'][1:]
ser_2.index = range(len(ser_2))
data['MA_12_PRED'] = ser_2

ser_3 = data['MA_26'][1:]
ser_3.index = range(len(ser_3))
data['MA_26_PRED'] = ser_3

ser_4 = data['boll_up'][1:]
ser_4.index = range(len(ser_4))
data['boll_up_PRED'] = ser_4

ser_5 = data['boll_down'][1:]
ser_5.index = range(len(ser_5))
data['boll_down_PRED'] = ser_5
data = data.fillna(0)
# data

In [10]:
data_new = data.drop(data.columns[36:],axis = 1)
data_new
data_new_array = np.array(data_new)
data_new_array.shape
from sklearn import preprocessing
min_max_scaler = preprocessing.MinMaxScaler()
data_new_array = min_max_scaler.fit_transform(data_new_array)
data_new_array
seq_new_5 = []
for j in range(len(data_new_array)):
    if j+5 < len(data_new_array):
        seq_new_5.append(data_new_array[j:j+5])
seq_new_5 = np.array(seq_new_5)
seq_new_5.shape

(35753, 5, 36)

In [11]:
test_data = seq_new_5[12000:14000]
test_data = np.array(test_data)
test_data.shape
data2 = data[12000:14000]
test_label = []
for i in data2['RaiseDown_1']:
    if i > 0.002:
        test_label.append([1,0,0,0,0])
    elif i > 0.0005:
        test_label.append([0,1,0,0,0])
    elif i> -0.0005:
        test_label.append([0,0,1,0,0])
    elif i > -0.002:
        test_label.append([0,0,0,1,0])
    else:
        test_label.append([0,0,0,0,1])
test_label = np.array(test_label)
test_label.shape
test_label_list = test_label.tolist()
# 测试集五类样本的数量
print u'大涨：', test_label_list.count([1,0,0,0,0])
print u'小涨：', test_label_list.count([0,1,0,0,0])
print u'平稳：', test_label_list.count([0,0,1,0,0])
print u'小跌：', test_label_list.count([0,0,0,1,0])
print u'大跌：', test_label_list.count([0,0,0,0,1])

大涨： 17
小涨： 320
平稳： 1304
小跌： 349
大跌： 10


In [183]:
n_input = 36 # 特征数量
n_steps = 5 # 时间序列长度
n_hidden = 500 # 隐藏层神经元个数
n_classes = 5 # 分类数量

tf.reset_default_graph()  # 重置流图   

xtr = tf.placeholder("float", [None, n_steps, n_input])
ytr = tf.placeholder("float", [None,n_classes])
# xtr1 = tf.placeholder("float", [None, n_input*n_steps])  
# xte1 = tf.placeholder("float", [n_input*n_steps])
pclass = tf.placeholder("float", None)
weights = {
    'out': tf.Variable(tf.random_normal([n_hidden, n_classes]))
}
biases = {
    'out': tf.Variable(tf.random_normal([n_classes]))
}
def RNN(x, weights, biases):

    # 更改数据集的格式，使其满足RNN网络的输入需求
    # 现在输入格式: (batch_size, n_steps, n_input)
    # 需要的格式: 'n_steps' 个tensor，每个的格式为 (batch_size, n_input)

    # 变换batch_size 和 n_steps的位置
    x = tf.transpose(x, [1, 0, 2])
    # 重新调整格式 (n_steps*batch_size, n_input)
    x = tf.reshape(x, [-1, n_input])
    # 分割成一系列的 'n_steps' 个tensors，每个的格式为 (batch_size, n_input)
    x = tf.split(0, n_steps, x)

    # 定义LSTM神经网络结构
    lstm_cell = rnn_cell.BasicLSTMCell(n_hidden, forget_bias=1.0)
    # 获取输出和状态
    outputs, states = rnn.rnn(lstm_cell, x, dtype=tf.float32)
    # 返回输出
    return tf.matmul(outputs[-1], weights['out']) + biases['out']
# 获取预测值
pred = RNN(xtr, weights, biases)

correct_pred = tf.equal(tf.argmax(pred, 1), tf.argmax(ytr, 1))
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

saver = tf.train.Saver()
# 创建会话
sess = tf.InteractiveSession()
saver.restore(sess, "/home/chocolate/Model_LSTM-Future_20161123/models/model_1min.ckpt")
print "Model restored"
test_len = 1
test_data1 = test_data[:test_len].reshape((-1, n_steps, n_input))
test_label1 = test_label[:test_len]
print("Testing Accuracy:",
        sess.run(accuracy, feed_dict={xtr: test_data1, ytr: test_label1}))
 


Model restored
('Testing Accuracy:', 1.0)


Exception AssertionError: AssertionError() in <bound method InteractiveSession.__del__ of <tensorflow.python.client.session.InteractiveSession object at 0x7faaf02e9f10>> ignored


In [186]:
# pred的每一行都是五维数组，根据最大隶属度关系，它的哪个位置的值最大，预测属于哪一类
pred1 = sess.run(pred,feed_dict={xtr: test_data1})
re = np.where(pred1 == np.max(pred1))
re

    


(array([0]), array([2]))

In [76]:
import tensorflow as tf  
a = tf.constant(5.0)  
b = tf.constant(6.0)  
c = a * b  
with tf.Session():  
# We can also use 'c.eval()' here.  
  print(c.eval()) 

30.0


In [110]:
print tf.argmax(pred, 1)

Tensor("ArgMax_2:0", shape=(?,), dtype=int64)


In [154]:
print pred

Tensor("add:0", shape=(?, 5), dtype=float32)


In [68]:
import tensorflow as tf  
import numpy as np  
      
    # save to file  
W = tf.Variable([[1,1,1],[2,2,2]],dtype = tf.float32,name='weight')  
b = tf.Variable([[1,2,3]],dtype = tf.float32,name='biases')  
      
init = tf.initialize_all_variables()  
saver = tf.train.Saver()  
with tf.Session() as sess:  
    sess.run(init)  
    save_path = saver.save(sess,"/home/chocolate/save_net.ckpt")  
    print ("save to path:",save_path)  

Exception AssertionError: AssertionError() in <bound method InteractiveSession.__del__ of <tensorflow.python.client.session.InteractiveSession object at 0x7faac666c990>> ignored


('save to path:', '/home/chocolate/save_net.ckpt')


In [69]:
import tensorflow as tf  
import numpy as np  
tf.reset_default_graph()  # 重置流图      
W = tf.Variable(np.arange(6).reshape((2,3)),dtype = tf.float32,name='weight')  
b = tf.Variable(np.arange(3).reshape((1,3)),dtype = tf.float32,name='biases')      
saver = tf.train.Saver()  
with tf.Session() as sess:  
    saver.restore(sess,"/home/chocolate/save_net.ckpt")  
    print ("weights:",sess.run(W))  
    print ("biases:",sess.run(b))  

('weights:', array([[ 1.,  1.,  1.],
       [ 2.,  2.,  2.]], dtype=float32))
('biases:', array([[ 1.,  2.,  3.]], dtype=float32))


In [40]:
import tensorflow as tf
v1 = tf.Variable(tf.random_normal([1,2]), name="v1")
v2 = tf.Variable(tf.random_normal([2,3]), name="v2")
init_op = tf.initialize_all_variables()
saver = tf.train.Saver()
with tf.Session() as sess:
    sess.run(init_op)
    saver_path = saver.save(sess, "/home/chocolate/save_net.ckpt")
    print "Model saved in file: ", saver_path

Model saved in file:  /home/chocolate/save_net.ckpt


In [42]:
import tensorflow as tf
tf.reset_default_graph()  # 重置流图
v1 = tf.Variable(tf.random_normal([1,2]), name="v1")
v2 = tf.Variable(tf.random_normal([2,3]), name="v2")
saver = tf.train.Saver()

with tf.Session() as sess:
    saver.restore(sess, "/home/chocolate/save_net.ckpt")
    print "Model restored"
    print ("weights:",sess.run(v1))  
    print ("biases:",sess.run(v2))  

Model restored
('weights:', array([[ 0.66326529,  0.04105425]], dtype=float32))
('biases:', array([[ 2.00193977,  1.52981889,  1.01274168],
       [-0.43986952, -0.477148  , -1.84927106]], dtype=float32))
